In [1]:
import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
import Solver3 as sl 
from scipy.interpolate import interp1d

%load_ext autoreload

%autoreload 2


# Generating Delta Vector

In [31]:

def cov_matrix(sigma, num_points):
  sigma = (sigma ** 2) * np.eye(num_points)
  return sigma


def compute_basis_functions_at_x(x, mesh):
    """
    Given an observation point x and the mesh, compute the two nonzero
    finite element basis (hat) functions at x.
    
    Returns:
      k: the index such that x is in [mesh[k], mesh[k+1]]
      psi_left: the value of the basis function associated with mesh[k] at x.
      psi_right: the value of the basis function associated with mesh[k+1] at x.
    """
    mesh = np.asarray(mesh, dtype=float)
    # Find the index k such that x is in [mesh[k], mesh[k+1]]
    k = np.searchsorted(mesh, x) - 1
    k = np.clip(k, 0, len(mesh) - 2)  # ensure k is valid
    
    psi_left = sl.phi_i(k, x, mesh)
    psi_right = sl.phi_i(k+1, x, mesh)
    return k, psi_left, psi_right

# Example: Evaluate the FE solution at a set of observation points
def fe_solution_at_obs(c_sol, mesh, x_obs):
    """
    Compute the finite element solution at observation points x_obs,
    given the nodal solution c_sol and the mesh.
    
    Parameters:
      c_sol : array of nodal values (length N)
      mesh  : array of node coordinates (length N)
      x_obs : array of observation points
      
    Returns:
      c_interp: array of interpolated FE solution values at x_obs.
    """
    c_sol_full = sl.assemble_nodal_values(c_sol)
    c_sol_full_array = np.asarray(c_sol_full).flatten()
    c_interp = np.zeros_like(x_obs, dtype=float)
    
    for idx, x in enumerate(x_obs):
        k, psi_left, psi_right = compute_basis_functions_at_x(x, mesh)
        # The solution at x is the weighted average of the two nodal values
        c_interp[idx] = c_sol_full_array[k] * psi_left + c_sol_full_array[k+1] * psi_right
    return c_interp

# Alternatively, if many observation points fall in the same element,
# you could vectorize the process by grouping x_obs by element.



In [32]:
def add_noise(observations_at_xi, num_points, sigma):
    """
    Adds a normally distributed noise, theta
    to observations from the forward solver.

    Arguments:
    observations_at_xi : observations at predetermined xi using interplotion. 
    num_points : how big your covariance matrix is 

    Returns:
    Delta : Array of Noisy observations.
    
    """
    sigma = cov_matrix(sigma, num_points)
    noise = np.random.multivariate_normal(np.zeros(num_points), sigma)
    delta = observations_at_xi + noise 
    return delta



# Define Likelihood Function 

In [33]:
def phi(observations, predicted, sigma, num_points) :
    '''
    For a set of predetermined points xi -- obtained via np.linspace,
    this function defines the likelihood function 

    Arguments:
    observations: Generated noisy observation using beta_true -- corresponds to y in literature
    predicted: For a proposed beta_i, we compute the noisy observation using the forward solver 
    -- corresponds to g(beta_i) in literature

    Returns: 
    Likelihood function that is proportional to the prior distribution
    
    '''
    covariance_matrix = cov_matrix(sigma, num_points)
    diff = predicted - observations
    covariance_matrix_inv = np.linalg.inv(covariance_matrix) 
    val = 0.5 * diff.T @ covariance_matrix_inv @ diff
    return val



In [34]:
def compute_A(phi_0, phi_i, sigma):

    val = np.exp(phi_0 - phi_i)
    
    return val

# MCMC algorithm 

In [40]:
def MCMC(beta_true, number_of_iter, burn_in, sigma, num_points): 
    '''
    Builds a Markov Chain 

    Key Steps:
    1. Initialise a choice of Beta, beta_0 
    2. Compute likelihood of beta_0, using delta and beta_0_predicted
    3. Initialise the loop.
        - we propose a new beta_i from x* ~ Uniform(0.15, 0.85) and r ~ Uniform(0, 0.15)
        - compute y_i and g(beta_i)
        - compute likelihood using {y_i and g(beta_i)}
        - set alpha = min{1, likelihood }
    '''
    # set seed 
    np.random.seed(72)
    # range of uniform distribution 
    x_star_range = (0.3, 0.7) 
    r_range = (0.1, 0.2) 
    chain = []
    ddof_list = []
    # compute delta 
    mesh_true , c_sol_true, ddof_true= sl.refinement_loop(0.00001, beta_true) 
    y_true = fe_solution_at_obs(c_sol_true, mesh_true, np.linspace(0.0, 1.0, num_points))
    delta = add_noise(y_true, num_points, sigma)

    # draw first copy of beta --> beta_0

    beta_0 = np.array([
            np.random.uniform(*x_star_range),
            np.random.uniform(*r_range)
        ])
    print("Beta_0:", beta_0)
    
    # initialise current observations and likelihood 
    mesh_0, c_sol_0, ddof_0 = sl.refinement_loop(0.001, beta = beta_0)
    y_0 = fe_solution_at_obs(c_sol_0, mesh_0, np.linspace(0.0, 1.0, num_points))
    phi_0 = phi(delta, y_0, sigma, num_points)
    print("phi_0:", phi_0)
    ddof_list.append((beta_0, ddof_0))
    
    iter_count = 0
    acceptance_count = 0 
    acceptance_prob_history = []

    for i in range(number_of_iter):
        beta_proposal = np.array([
            np.random.uniform(*x_star_range),
            np.random.uniform(*r_range)
        ])
        print("beta proposal:", beta_proposal)
        mesh_proposal, c_sol_proposal, ddof_proposal = sl.refinement_loop(0.001, beta = beta_proposal)
        y_proposal = fe_solution_at_obs(c_sol_proposal, mesh_proposal, np.linspace(0.0, 1.0, num_points))
        ddof_list.append((beta_proposal, ddof_proposal))
        phi_proposal = phi(delta, y_proposal, sigma, num_points)
        print("phi proposal:", phi_proposal)
        # compute acceptance probability 
        A = compute_A(phi_0, phi_proposal, sigma)
        acceptance_prob = min(1, A)
        acceptance_prob_history.append(acceptance_prob)
        print("acceptance probablity:", acceptance_prob)
        
        # Accept or reject the proposal
        if np.random.rand() < acceptance_prob:
            beta_0 = beta_proposal # update the current state as the last accepted proposal
            y_0 = y_proposal # update the current observations to the last accepted observation
            phi_0 = phi_proposal
            acceptance_count += 1
        
        # Record the current state.
        chain.append(beta_0.copy())
        print("Chain length:", len(chain))
        
    
    chain = np.array(chain)
    # Compute the MCMC estimate as the mean of the samples after burn-in.
    beta_mcmc = np.mean(chain[burn_in:], axis=0)
    
    return chain, beta_mcmc, acceptance_prob_history, acceptance_count, ddof_list

In [41]:
number_of_iter = 10000
burn_in = 5000
sigma = 0.01
num_points = 100
beta_true = np.array([0.65, 0.15])

chain, beta_mcmc, acceptance_history, acceptance_count, ddof_list = MCMC(beta_true, number_of_iter, burn_in, sigma, num_points)
print("True beta:", beta_true )
print("MCMC estimated beta:", beta_mcmc)
print("Acceptance Probability History:", acceptance_history)
print("Acceptance Count:", acceptance_count)
print("ddof list:", ddof_list)


Beta_0: [0.42297268 0.15835503]
phi_0: 68.008265758628
beta proposal: [0.54983213 0.13747841]
phi proposal: 71.33818347125305
acceptance probablity: 0.03579605050949741
Chain length: 1
beta proposal: [0.67185419 0.15922371]
phi proposal: 56.20827730555973
acceptance probablity: 1
Chain length: 2
beta proposal: [0.31719453 0.18251206]
phi proposal: 82.51310470232974
acceptance probablity: 3.76667902929509e-12
Chain length: 3
beta proposal: [0.59148871 0.19615747]
phi proposal: 54.182782804847804
acceptance probablity: 1
Chain length: 4
beta proposal: [0.43403773 0.1302069 ]
phi proposal: 73.01430762857684
acceptance probablity: 6.6309031273055915e-09
Chain length: 5
beta proposal: [0.65700262 0.12432433]
phi proposal: 55.22974072473552
acceptance probablity: 0.351003908627385
Chain length: 6
beta proposal: [0.31051901 0.15495484]
phi proposal: 73.749086043259
acceptance probablity: 3.1802588680791713e-09
Chain length: 7
beta proposal: [0.41264299 0.1605062 ]
phi proposal: 67.48858484059

In [30]:
mean_beta = np.mean(chain, axis=0)
mean_beta_sq = np.mean(chain**2, axis=0)
var_beta = mean_beta_sq - mean_beta**2
var_beta

array([0.00192526, 0.00083297])

# Heat Map of Beta potential 

In [9]:
# import numpy as np
# import matplotlib.pyplot as plt
# import plotly.graph_objects as go

# # Parameters for the parameter grid (x* and r)
# x_star_vals = np.linspace(0.3, 0.7, 20)   # More candidate x* values for finer resolution
# r_vals = np.linspace(0.1, 0.2, 5)          # More candidate r values

# # Create a mesh grid of parameter pairs
# X, R = np.meshgrid(x_star_vals, r_vals)

# # Global parameters (use the same parameters throughout)
# epsilon = 1e-3       # Refinement tolerance
# sigma = 0.01         # Noise standard deviation
# num_points = 100     # Number of observation points

# # Generate the "true" observation using beta_true
# beta_true = np.array([0.65, 0.15])
# mesh_true, c_sol_true, ddof = sl.refinement_loop(epsilon, beta_true)
# y_true = fe_solution_at_obs(c_sol_true, mesh_true, np.linspace(0.0,1.0, num_points))
# observations = y_true + add_noise(y_true, num_points, sigma)
# true_potential = phi(y_true, y_true, sigma, num_points)

# # Prepare an array to store the misfit potential values
# potential = np.zeros_like(X)

# # Loop over the grid of (x*, r) candidates
# for i in range(X.shape[0]):
#     for j in range(X.shape[1]):
#         beta_candidate = np.array([X[i, j], R[i, j]])
#         mesh_candidate, c_sol_candidate, ddof = sl.refinement_loop(epsilon, beta_candidate)
#         y_candidate = fe_solution_at_obs(c_sol_candidate, mesh_candidate, np.linspace(0.0, 1.0, num_points))
#         potential[i, j] = phi(y_true, y_candidate, sigma, num_points)

# # Create the 2D contour plot with Plotly
# fig = go.Figure(data=[
#     go.Contour(
#         x=x_star_vals,
#         y=r_vals,
#         z=potential,
#         colorscale='turbid',
#         contours=dict(showlabels=True),
#         colorbar=dict(title="Potential")
#     ),
#     go.Scatter(
#         x=[beta_true[0]],
#         y=[beta_true[1]],
#         mode='markers',
#         marker=dict(size=10, color='cyan'),
#         name='True beta'
#     )
# ])

# # Update layout with axis titles and overall title
# fig.update_layout(
#     title='2D Contour Plot of the Misfit Potential',
#     xaxis_title='x*',
#     yaxis_title='r',
#     width=1000,
#     height=1000
# )

# fig.show()


In [11]:
from concurrent.futures import ProcessPoolExecutor

def run_single_chain(seed, beta_true, number_of_iter, burn_in, sigma, num_points):
    # Set the random seed to ensure independent chains.
    np.random.seed(seed)
    return MCMC(beta_true, number_of_iter, burn_in, sigma, num_points)

def run_multiple_chains(n_chains, beta_true, number_of_iter, burn_in, sigma, num_points):
    seeds = np.random.randint(0, 10000, size=n_chains)
    chains = []
    beta_mcmcs = []
    acceptance_histories = []
    acceptance_counts = []
    
    # Run chains in parallel using ProcessPoolExecutor
    with ProcessPoolExecutor() as executor:
        # Submit all the chains concurrently
        futures = [
            executor.submit(run_single_chain, seed, beta_true, number_of_iter, burn_in, sigma, num_points)
            for seed in seeds
        ]
        # Collect results as they complete
        for future in futures:
            chain, beta_mcmc, acceptance_prob_history, acceptance_count = future.result()
            chains.append(chain)
            beta_mcmcs.append(beta_mcmc)
            acceptance_histories.append(acceptance_prob_history)
            acceptance_counts.append(acceptance_count)
    
    return chains, beta_mcmcs, acceptance_histories, acceptance_counts

if __name__ == '__main__':
    n_chains = 4
    number_of_iter = 10000
    burn_in = 5000
    sigma = 0.01 
    num_points = 100
    beta_true = np.array([0.5, 1/6])
    chains, beta_mcmcs, acceptance_histories, acceptance_counts = run_multiple_chains(
        n_chains, beta_true, number_of_iter, burn_in, sigma, num_points
    )
print("True beta:", beta_true )
print("MCMC estimated beta:", beta_mcmcs)
print("Acceptance Probability History:", acceptance_histories)
print("Acceptance Count:", acceptance_counts)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.